In [19]:
%run ../../src/feature_selection_scripts/Feature_Selection_Bin_Cla_SRV.ipynb

['Seq', 'sTos', 'sTtl', 'Offset', 'sMeanPktSz', 'dMeanPktSz', 'AckDat', 'icmp', 'tcp', 'udp', 'ECO', 'FIN', 'INT', 'RST', 'Start']
(911917, 15)


In [20]:
from sklearn.ensemble import VotingClassifier
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow.keras.metrics import Precision, Recall


In [21]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

y_attack_type_train_encoded = encoder.fit_transform(y_attack_type_train)
y_attack_type_train_encoded = to_categorical(y_attack_type_train_encoded)

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input

num_classes = y_attack_type_train_encoded.shape[1]
num_features = X_train.shape[1]
sequence_length = 1



In [23]:
X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [24]:
model = Sequential([
    LSTM(50, input_shape=(1, X_train.shape[1])),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', Precision(), Recall()])

# Attempt to train again
model.fit(X_train_reshaped, y_attack_type_train_encoded, epochs=20, batch_size=32)

Epoch 1/20


/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


28498/28498 ━━━━━━━━━━━━━━━━━━━━ 19s 636us/step - accuracy: 0.9424 - loss: 0.2219 - precision: 0.9638 - recall: 0.9061
Epoch 2/20
28498/28498 ━━━━━━━━━━━━━━━━━━━━ 17s 611us/step - accuracy: 0.9895 - loss: 0.0393 - precision: 0.9901 - recall: 0.9892
Epoch 3/20
28498/28498 ━━━━━━━━━━━━━━━━━━━━ 17s 613us/step - accuracy: 0.9926 - loss: 0.0269 - precision: 0.9931 - recall: 0.9922
Epoch 4/20
28498/28498 ━━━━━━━━━━━━━━━━━━━━ 17s 610us/step - accuracy: 0.9937 - loss: 0.0223 - precision: 0.9941 - recall: 0.9934
Epoch 5/20
28498/28498 ━━━━━━━━━━━━━━━━━━━━ 17s 607us/step - accuracy: 0.9943 - loss: 0.0195 - precision: 0.9947 - recall: 0.9941
Epoch 6/20
28498/28498 ━━━━━━━━━━━━━━━━━━━━ 17s 612us/step - accuracy: 0.9947 - loss: 0.0177 - precision: 0.9950 - recall: 0.9944
Epoch 7/20
28498/28498 ━━━━━━━━━━━━━━━━━━━━ 17s 608us/step - accuracy: 0.9950 - loss: 0.0163 - precision: 0.9953 - recall: 0.9948
Epoch 8/20
28498/28498 ━━━━━━━━━━━━━━━━━━━━ 18s 615us/step - accuracy: 0.9954 - loss: 0.0154 - precis

In [25]:
import pickle
with open('../GradientBoosting/gradient_boosting.pkl', 'rb') as f:
    gb_clf = pickle.load(f)

In [26]:
y_pred_binary = gb_clf.predict(X_test)

In [27]:
X_malicious_test = X_test[y_pred_binary == 1]

y_attack_type_test_malicious = y_attack_type_test[y_pred_binary == 1]
y_attack_type_test_malicious_encoded = encoder.transform(y_attack_type_test_malicious)
y_attack_type_test_malicious_encoded = to_categorical(y_attack_type_test_malicious_encoded, num_classes=num_classes)

In [28]:
X_malicious_test_reshaped = X_malicious_test.reshape((X_malicious_test.shape[0], 1, X_malicious_test.shape[1]))

In [29]:
y_pred_attack_type = model.predict(X_malicious_test_reshaped)

loss, accuracy, precision, recall = model.evaluate(X_malicious_test_reshaped, y_attack_type_test_malicious_encoded)
print(f'Model accuracy on malicious test data: {accuracy*100:.2f}%')
print(f'Precision: {precision*100:.2f}%')
print(f'Recall: {recall*100:.2f}%')

# If you need the F1-score, you can calculate it from the precision and recall:
f1_score = 2 * (precision * recall) / (precision + recall)
print(f'F1 Score: {f1_score*100:.2f}%')

5755/5755 ━━━━━━━━━━━━━━━━━━━━ 2s 354us/step
5755/5755 ━━━━━━━━━━━━━━━━━━━━ 2s 356us/step - accuracy: 0.9973 - loss: 0.0098 - precision: 0.9974 - recall: 0.9972
Model accuracy on malicious test data: 99.73%
Precision: 99.75%
Recall: 99.72%
F1 Score: 99.73%


In [30]:
import keras
keras.saving.save_model(model, 'LSTM_model.keras')